In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np

from promptopt import models
from promptopt import datasets
from promptopt import rlhf

from matplotlib import pyplot as plt
import matplotlib as mpl

In [ ]:
n_embedding_dims = 2

In [ ]:
net_arch_kwargs = {
  'n_layers': 0,
  'layer_size': None
}
pref_model = models.PrefModel(n_embedding_dims, net_arch_kwargs=net_arch_kwargs)

In [ ]:
pref_model_train_config = rlhf.get_default_config()
pref_model_train_config.verbose = True

In [ ]:
n_embeddings = 100
embeddings = np.random.normal(0, 1, (n_embeddings, n_embedding_dims))

In [ ]:
embedding_dataset = datasets.EmbeddingDataset(embeddings=list(embeddings))

In [ ]:
pref_dataset = datasets.PrefDataset(embedding_dataset)

In [ ]:
R = np.sum(embeddings, axis=1)
for i in range(n_embeddings):
  for j in range(i+1, n_embeddings):
    pref = (R[i] > R[j]).astype(int)
    pref_dataset.append(i, j, pref)

In [ ]:
plt.scatter(embeddings[:, 0], embeddings[:, 1], c=R)
plt.show()

In [ ]:
optimizer = rlhf.RLHF(pref_model, pref_model_train_config, embedding_dataset, pref_dataset)

In [ ]:
embedding = optimizer.max_embedding(reg_const=1e2)

In [ ]:
optimizer.predict_prefs(embedding)